[Reference](https://medium.com/plumbersofdatascience/a-python-data-pipeline-to-track-holiday-spending-with-mint-api-bigquery-and-tableau-9ae7f1e2e77f)

In [1]:
pip install mintapi

     |████████████████████████████████| 958 kB 6.6 MB/s 
     |████████████████████████████████| 87 kB 5.4 MB/s 
     |████████████████████████████████| 904 kB 46.6 MB/s 
     |████████████████████████████████| 62 kB 733 kB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 3.6 MB 43.0 MB/s 
  Created wheel for mintapi: filename=mintapi-1.61-py3-none-any.whl size=25276 sha256=0335639ccfd40956f31dd227b5a5fdf1e820464ba72b4d02a2474e31cd8e0863
  Stored in directory: /root/.cache/pip/wheels/c1/63/44/5d3038eb3e9d414941f010df1675f34c0bc8db53257e37bad1
Successfully built mintapi
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests

# Extract


In [2]:
import pandas as pd 
from google.cloud import bigquery
import mintapi as mintapi
# Opens Mint in new browser window through Selenium webdriver. 
mint_auth = mintapi.Mint(
    'email',
    'password',
    mfa_method = 'sms',
    mfa_input_callback=None,
    mfa_token=None,
    intuit_account=None,
    headless=False,
    session_path=None,
    imap_account=None,
    imap_password=None,
    imap_server=None,
    imap_folder='INBOX',
    wait_for_sync=False, 
    wait_for_sync_timeout=300,
    use_chromedriver_on_path=False
    )

In [3]:
# Retrieves all accounts 
mint_auth.get_accounts()
# Retrieves budgetary information 
mint_auth.get_budgets()
# Retrieves transaction information 
mint_auth.get_transactions()
mint_auth.get_transactions_csv()
mint_auth.get_transactions_json()
# Retrieves current net worth (one value).
mint_auth.get_net_worth()
# Retrieves current credit score (one value).
mint_auth.get_credit_score()
# Retrieves billing information 
mint_auth.get_bills()
# Retrieves investment information as JSON 
mint_auth.get_invests_json()

# Transform


In [4]:
# Get mint transactions as a dataframe. 
mint = mint_auth.get_transactions(include_investment=True)
# Close webdriver connection. 
mint_auth.close()
# Clean dataframe, format columns & add features. 
mint_df = pd.DataFrame(mint)
mint_df = mint_df.drop('labels', axis=1) 
mint_df = mint_df.drop('notes', axis=1)
mint_df['description'] = mint_df['description'].str.title()
mint_df['original_description'] = mint_df['original_description'].str.title()
mint_df['transaction_type'] =mint_df['transaction_type'].str.title()
mint_df['category'] = mint_df['category'].str.title()
mint_df['paycheck_amount_remaining_week'] = round(**** - mint_df['amount'],2)
mint_df['paycheck_amount_remaining_month'] = round(**** - mint_df['amount'],2)
mint_df['discretionary_remaining_month'] = round(**** - mint_df['amount'],2)
mint_df['discretionary_remaining_year'] = round(***** - mint_df['amount'],2)
mint_df 

# Load


In [5]:
client = bigquery.Client()
dataset_id = 'mint'
table_id = 'transactions'
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
    job_config.write_disposition='WRITE_TRUNCATE'
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.autodetect=True
    job_config.ignore_unknown_values=True
job = client.load_table_from_dataframe(
    mint_df,
    table_ref,
    location='US',
    job_config=job_config)
    
job.result()
return 'Transactions table loaded.'